In [1]:
import tensorflow as tf
import cv2
import keras
import os
import pandas as pd
import matplotlib.pyplot as plt
import numpy as np
from keras.callbacks import ModelCheckpoint
import pickle

2024-06-22 11:59:25.917951: I tensorflow/core/platform/cpu_feature_guard.cc:210] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: SSE4.1 SSE4.2 AVX AVX2 AVX512F FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.


In [41]:
# Data Augmentations
def fliplr(image):
    image = cv2.flip(image, 1)
    return image

def flipud(image):
    image = cv2.flip(image, 0)
    return image

In [ ]:
target_height, target_width = 80, 80
number_of_classes = 2

In [4]:
def load_and_preprocess2(image_path):
    img = cv2.imread(image_path) # 3 channel image
    if img is None:
        print('Error loading image: ', f'{image_path}')
        return None
    print("Original image size: ", f'{img.shape}')
    image = cv2.imread(image_path, cv2.COLOR_BGR2LAB)
    crop = img
    L, a, b = cv2.split(image)
    kernel_size = 20

    kernel_v = np.zeros((kernel_size, kernel_size))
    kernel_h = np.copy(kernel_v)
    kernel_d1 = np.copy(kernel_v)
    kernel_d2 = np.copy(kernel_v)

    kernel_v[:, int((kernel_size - 1)/2)] = np.ones(kernel_size)
    kernel_h[int((kernel_size - 1)/2), :] = np.ones(kernel_size)

    for i in range(kernel_size):
        kernel_d1[i, i] = 1

    for i in range(kernel_size):
        kernel_d2[i, kernel_size - i - 1] = 1

    kernel_v /= kernel_size
    kernel_h /= kernel_size
    kernel_d1 /= kernel_size
    kernel_d2 /= kernel_size

    vertical_mb = cv2.filter2D(b, -1, kernel_v)

    horizonal_mb = cv2.filter2D(b, -1, kernel_h)

    d1_mb = cv2.filter2D(b, -1, kernel_d1)

    d2_mb = cv2.filter2D(b, -1, kernel_d2)

    combined_blur = cv2.addWeighted(vertical_mb, 0.25, horizonal_mb, 0.25, 0)
    combined_blur = cv2.addWeighted(combined_blur, 0.5, d1_mb, 0.25, 0)
    img = cv2.addWeighted(combined_blur, 0.5, d2_mb, 0.25, 0)

    gimg = img
    max_brightness_pixel = np.unravel_index(np.argmax(gimg), gimg.shape)
    max_brightness_x, max_brightness_y = max_brightness_pixel

    padding_size = 70
    start_y = max(0, max_brightness_y - padding_size)
    end_y = min(img.shape[0], max_brightness_y + padding_size)
    start_x = max(0, max_brightness_x - padding_size)
    end_x = min(img.shape[1], max_brightness_x + padding_size)

    crop = crop[start_x:end_x, start_y:end_y ]

    img = cv2.resize(crop, (target_height, target_width))
    img = img.astype('float32') / 255 # Normalize pixel values
    return img